In [1]:
from ag.layers import *
from ag.models import *
from ag.utils  import *
from ag.setups import *

from lra.layers import AMGOLU, GatedOrthoKernel, HeadWiseFF

In [2]:
ag_train, ag_valid, ag_test, tokenizer, b_model = get_data_tokenizer_bert(AG_SETUP)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Using custom data configuration default
Reusing dataset ag_news (/home/yorshula/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29

  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /home/yorshula/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548/cache-af9af74624e7d37b.arrow
Loading cached processed dataset at /home/yorshula/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548/cache-7c422186a4414477.arrow
Loading cached processed dataset at /home/yorshula/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548/cache-fd92ef3d539dd8d5.arrow
Loading cached processed dataset at /home/yorshula/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548/cache-6acea5ac1d887497.arrow
Loading cached processed dataset at /home/yorshula/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548/cache-5df52efc21155600.arrow
Loading cached processed 

In [3]:
#AG_SETUP['model_type'] = AGTransformerSkip

def att_factory(hidden_dim, qkv_dim, num_heads, dropout_rate):
    
    lka = nn.Sequential(
        AMGOLU(num_heads, qkv_dim, qkv_dim // num_heads // 4, dropout_rate, nn.Sigmoid(), nn.Identity(), False, LAMBDA=0.0),
        AMGOLU(num_heads, qkv_dim, qkv_dim // num_heads // 4, dropout_rate, nn.Sigmoid(), nn.Identity(), False, LAMBDA=0.0),
        AMGOLU(num_heads, qkv_dim, qkv_dim // num_heads // 4, dropout_rate, nn.Sigmoid(), nn.Softplus(), False, LAMBDA=0.0),
        
        #GatedOrthoKernel(num_heads, hidden_dim, dropout_rate, nn.Sigmoid(), nn.Identity(), False, LAMBDA=0.1),
        #GatedOrthoKernel(num_heads, hidden_dim, dropout_rate, nn.Sigmoid(), nn.Identity(), False, LAMBDA=0.1),
        #GatedOrthoKernel(num_heads, hidden_dim, dropout_rate, nn.Sigmoid(), nn.Softplus(), False, LAMBDA=0.1)

        #HeadWiseFF(num_heads, qkv_dim, dropout_rate, nn.Softplus(), use_bias=False, LAMBDA=0.1),   
    )
    return LKAAttention(hidden_dim, qkv_dim, num_heads, dropout_rate, lka)
    
    #return FtAttention(hidden_dim, qkv_dim, num_heads, dropout_rate)
    #return SimpleAttention(hidden_dim, qkv_dim, num_heads, dropout_rate, use_lin=True)
    #return SimpleAttention(hidden_dim, qkv_dim, num_heads, dropout_rate, use_lin=False)

model, criterion, optimizer, schedule_func, scheduler = coarse_training_setup(AG_SETUP, tokenizer, weight_donor=b_model, attention_factory=att_factory)

Unparsable param: token_type_embeddings.weight
Cannot check param: blocks.0.attention.lka.0.orth_weight.weight
Cannot check param: blocks.0.attention.lka.0.gate_weight_a.weight
Cannot check param: blocks.0.attention.lka.0.gate_weight_b.weight
Cannot check param: blocks.0.attention.lka.1.orth_weight.weight
Cannot check param: blocks.0.attention.lka.1.gate_weight_a.weight
Cannot check param: blocks.0.attention.lka.1.gate_weight_b.weight
Cannot check param: blocks.0.attention.lka.2.orth_weight.weight
Cannot check param: blocks.0.attention.lka.2.gate_weight_a.weight
Cannot check param: blocks.0.attention.lka.2.gate_weight_b.weight
Cannot check param: blocks.1.attention.lka.0.orth_weight.weight
Cannot check param: blocks.1.attention.lka.0.gate_weight_a.weight
Cannot check param: blocks.1.attention.lka.0.gate_weight_b.weight
Cannot check param: blocks.1.attention.lka.1.orth_weight.weight
Cannot check param: blocks.1.attention.lka.1.gate_weight_a.weight
Cannot check param: blocks.1.attention.

In [4]:
model, criterion, optimizer, schedule_func, scheduler = fine_training_setup(AG_SETUP, model)

In [5]:
import torch

AG_SETUP['device'] = 'cuda:1'

AG_SETUP['coarse_periods'] = 1
AG_SETUP['coarse_eval_period'] = 10

AG_SETUP['fine_periods'] = 1
AG_SETUP['fine_eval_period'] = 10

test_accuracy = [  ]

for i in range(1): ####!!!!!!!!!!!!!!
  path = 'model_to_test_' + str(i) + '.b'

  #Tune only learnable kernels + embeddings + cls head
  model, criterion, optimizer, schedule_func, scheduler = coarse_training_setup(AG_SETUP, tokenizer, weight_donor=b_model, attention_factory=att_factory)

  checkpoint = train_ag_model(AG_SETUP, model, path, ag_train, ag_valid, optimizer, criterion, scheduler,
                             AG_SETUP['coarse_periods'], AG_SETUP['coarse_eval_period'], AG_SETUP['coarse_skip_eval'])
  model.load_state_dict(checkpoint['model_state_dict'])

  #Fine-tune the rest
  model, criterion, optimizer, schedule_func, scheduler = fine_training_setup(AG_SETUP, model)
  checkpoint = train_ag_model(AG_SETUP, model, path, ag_train, ag_valid, optimizer, criterion, scheduler,
                             AG_SETUP['coarse_periods'], AG_SETUP['coarse_eval_period'], AG_SETUP['coarse_skip_eval'])
  model.load_state_dict(checkpoint['model_state_dict'])
    
  _, _, acc = test_ag_model(AG_SETUP, model, criterion, ag_test)
  test_accuracy.append(acc)

test_accuracy = np.mean(test_accuracy)

print(f'\nTotal accuracy: {test_accuracy:.4f}')

Unparsable param: token_type_embeddings.weight
Cannot check param: blocks.0.attention.lka.0.orth_weight.weight
Cannot check param: blocks.0.attention.lka.0.gate_weight_a.weight
Cannot check param: blocks.0.attention.lka.0.gate_weight_b.weight
Cannot check param: blocks.0.attention.lka.1.orth_weight.weight
Cannot check param: blocks.0.attention.lka.1.gate_weight_a.weight
Cannot check param: blocks.0.attention.lka.1.gate_weight_b.weight
Cannot check param: blocks.0.attention.lka.2.orth_weight.weight
Cannot check param: blocks.0.attention.lka.2.gate_weight_a.weight
Cannot check param: blocks.0.attention.lka.2.gate_weight_b.weight
Cannot check param: blocks.1.attention.lka.0.orth_weight.weight
Cannot check param: blocks.1.attention.lka.0.gate_weight_a.weight
Cannot check param: blocks.1.attention.lka.0.gate_weight_b.weight
Cannot check param: blocks.1.attention.lka.1.orth_weight.weight
Cannot check param: blocks.1.attention.lka.1.gate_weight_a.weight
Cannot check param: blocks.1.attention.